# Objective

- Implement a task-oriented collaborative workflow of agents to achieve a business objective using `crewai`

# Setup

## Installation

In [2]:
%pip install -q crewai langchain langchain-groq sentence-transformers langchain-huggingface


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Imports

In [3]:
import json, os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew, Process, LLM

load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']

In [ ]:
# with open('config-azure.json') as f:
#     configs = f.read()
# creds = json.loads(configs)

In [4]:
# default_llm = ChatGroq(
#     model='gemma2-9b-it',
#     api_key=groq_api_key,
#     temperature=0
# )

default_llm = LLM(model='groq/gemma2-9b-it', api_key=groq_api_key)

# Business Objective: Lead Scoring

Consider a tax software company that sells accounting and tax planning products for its enterprise customers. They have two core products - TaxEdge Pro and TaxEdge Analytics. The company wants to implement a lead scoring system for the B2B leads that show interest in the company by filling in a lead form on their website.

To score leads, the company wants to use its ideal customer profile, the products it has and match them to the customer needs as mentioned in lead form responses. The company wants to make lead followup more effective by reaching out personally only to those leads who have a high chance of conversion.

# Solution Implementation

The process of scoring form responses from leads follows a clear business process and hence is apt for a crew of agents approach. Tasks that form this business process are:
1. Extract key information from the form response.
2. Research the leads' background, industry and outlook and augment information in the form response.
3. Score the lead using the information in the form and the research conducted in step 2.
4. Prepare a plan for a company representative to approach the lead if they are deemed to qualify for a follow-up call.

For these tasks that we need to execute in sequence, we attach a specialized agent to each task and assemble a crew.

## Let us now implement this plan using abstractions from `crew.ai`.

**Inputs**

In [5]:
company = 'TaxEdge Solutions'
product_names = 'TaxEdge Pro & TaxEdge Analytics'

product_descriptions = """
TaxEdge Pro is a comprehensive tax management software designed to simplify corporate tax filing and compliance processes. It offers features such as automated tax calculations, integrated payroll tax management, and seamless reporting tools to ensure businesses remain compliant with ever-changing tax laws.
TaxEdge Analytics is an advanced reporting and insights tool that helps businesses gain a deeper understanding of their tax data. It enables businesses to analyze tax trends, optimize deductions, and generate customized reports to support better financial decision-making.
"""

ideal_customer_profile_description = """
The ideal customers for TaxEdge Solutions are medium to large enterprises, including accounting firms and corporate finance departments, that manage complex tax filings and compliance requirements. These customers value efficiency, accuracy, and robust reporting tools to meet both regulatory demands and internal auditing standards. They typically face challenges such as staying up-to-date with tax law changes, managing multiple tax jurisdictions, and reducing the time spent on tax preparation and reporting.
"""

form_response = """
**Form Response**

1. **Name of your organization:** Alpha Financial Services Inc.
2. **Contact person's name and designation:** Rachel Green, Tax Compliance Manager
3. **Email address and phone number:** rachel.green@alphafinancial.com, (123) 456-7890
4. **What is the size of your company?** 500 employees
5. **How many employees are involved in tax-related processes?** 12 employees
6. **Do you currently use any tax management software? If yes, please specify.** Yes, we currently use TaxWise but are exploring alternatives for better reporting and analytics.
7. **What are your biggest challenges in tax compliance and reporting?** Keeping up with changing tax regulations across multiple jurisdictions, ensuring accuracy in tax calculations, and reducing the time spent on preparing quarterly reports.
8. **Which product are you most interested in: TaxEdge Pro, TaxEdge Analytics, or both?** Both products
9. **Would you like to schedule a demo or consultation?** Yes, I would like to schedule a demo next week.
"""

## Agents

The first step is to assemble individual agents specialized in each of the tasks identified to be executed to achieve the business objective. We define three agents below:
- Lead Analysis Agent: Analyzes form responses and extracts information needed for research and scoring
- Research Agent: Conducts research on the leads' company and industry. This research could use external tools such as search engines.
- Scoring and Planning Agent: Computes a lead score and assembles a product pitch inorder to position the correct product to the customer based on form response information and research conducted by the research agent.

Let us now define these agents using abstractions provided by `crewai`.

In [6]:
lead_analysis_agent = Agent(
    role="Lead Analysis Expert",
    goal=f"""
    You work at {company}, where you main goal is to analyze leads form responses
    to extract essential data for further research and scoring.
    """,
    backstory="""
    As a Lead Analysis Expert, you have an eye for detail and a knack for
    identifying key information. With your analytical skills, you sift through
    form responses to prepare data for in-depth research, setting the stage for
    effective lead qualification.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

research_agent = Agent(
    role="Industry Research Specialist",
    goal="""
    Conduct preliminary research on the lead's industry, company size, and AI
    use case to provide a solid foundation for scoring.
    """,
    backstory=f"""
    You're an Industry Research Specialist at {company} who thrives on digging
    deep into market trends, company backgrounds, and potential use cases for AI.
    Your research uncovers vital information that informs the scoring and
    planning process, ensuring a tailored approach to each lead.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

scoring_and_planning_agent = Agent(
    role="Lead Scoring and Strategic Planner",
    goal="""
    Utilize analyzed data and research findings to score leads and suggest an
    appropriate plan.
    """,
    backstory=f"""
    As the Lead Scoring and Strategic Planner at {company} you combine insights
    from lead analysis and research to score leads accurately and align them with the
    optimal offering. Your strategic vision and scoring expertise ensure that
    potential leads are matched with solutions that meet their specific needs.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

Each of these agents has a role, a goal and a backstory that together define the system message for the LLM attached to the agent.

## Tasks

With the agents in place, we can now design the tasks these agents should execute in order. These tasks are instances of the `Task` class from `crewai` and further describe the specific task in detail adding in specifics of the company and its products. Each task also has a clear direction on the expected output from the agent executing the task.

In [7]:
lead_analysis_task = Task(
  description=f"""
   Analyze the lead's form responses to extract critical information that
   might be useful for scoring them, planning a strategy to pitch them,
   or anything that might be helpful for taking next steps on selling
   them our {product_names}.

   We at {company} are more focused on {ideal_customer_profile_description}.

   Let's not spend much time in leads that are not a good fit for us.
   Short answers from leads are a yellow flag

   Lead Form Responses:
   {form_response}

   Product Descriptions:
   {product_descriptions}
  """,
  expected_output="""
  Highlighting key implict or explicit information from the lead's form
  responses that can be used for further research and scoring.
  We are looking for insights that can help us understand the lead's
  industry, company, and potential use cases.
  """,
  agent=lead_analysis_agent
)


research_task = Task(
    description=f"""
    Using the information gathered from the lead's data, conduct preliminary
    research on the lead's industry, company background, and potential use
    cases for {product_names}.
    Focus on finding relevant data that can aid in scoring the lead and
    planning a strategy to pitch them {product_names}.

    Let's not spend much time in leads that are not a good fit for us.
    Short answers from leads are a yellow flag.

    Lead Form Responses:
    {form_response}

    Product Descriptions:
    {product_descriptions}
    """,
    expected_output=f"""
    A research report that includes:
    - Industry overview and trends relevant to the lead's company.
    - Company-specific information like size, market position, and recent news.
    - Potential {product_names} use cases and benefits specific to the lead's industry and company.
    - Any other insights that would aid in scoring and planning.
    The report should be concise and actionable.
    """,
    agent=research_agent,
    output_file='research-report.md'
)


scoring_and_planning_task = Task(
    description=f"""
    Utilize the context and the lead's form response to score the lead.
    Consider factors such as industry relevance, company size, specific {product_names}
    use case potential, and readiness for partnership.
    Take into account the wording used on the form response and how lenghty
    the response was to score the lead.
    Based on the score, suggest the most appropriate strategy for the lead.
    Integrate this information to outline how {company} and {product_names} can
    specifically benefit this lead.

    We are more focused on {ideal_customer_profile_description}

    Let's not spend much time in leads that are not a good fit for us.
    Smaller comapnies will want to pay less.
    Short answers from leads are a yellow flag.
    Let's be a little pessimistic and focus the high scores on deals that are
    more likely to close and truly great.

    Lead Form Responses:
    {form_response}


    Product Descriptions:
    {product_descriptions}
    """,
    expected_output=f"""
    A comprehensive evaluation report for each lead that includes:
    - A score from 0 to 10
    - A tailored summary of how {company} and {product_names} can assist the lead.
    - A list of talking points and ideas for an initial email and meeting, personalized for the lead.
    This output should serve as a detailed guide for engaging with the lead effectively.
    """,
    agent=scoring_and_planning_agent,
    output_file='report.md'
)

Once the final task is executed, the output needs to be presented in a markdown format and written to a report.md file.

We can now instantiate a crew with these agents by defining the set of agents and a set of tasks that need to be solved in a sequence.

In [8]:
lead_assessment_crew = Crew(
  agents=[lead_analysis_agent, research_agent, scoring_and_planning_agent],
  tasks=[lead_analysis_task, research_task, scoring_and_planning_task],
  process=Process.sequential,
  verbose=True
)

In [9]:
# Begin the task execution
response = lead_assessment_crew.kickoff()
print(response.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e9c46c4b-d65f-4d9c-947a-9aa7af0c38e5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Analysis Expert                                                                                    │
│                                                                                                                 │
│  Task:                                                                                                          │
│     Analyze the lead's form responses to extract critical information that                                      │
│     might be useful for scoring them, planning a strategy to pitch them,                                        │
│     or anything that might be helpful for taking next steps on selling                                          │
│     them our TaxEdge Pro & TaxEdge Analytics.                                                                   │
│                                                                                                                 │
│     We at TaxEdge Solutions are more focused on                                                                 │
│  The ideal customers for TaxEdge Solutions are medium to large enterprises, including accounting firms and      │
│  corporate finance departments, that manage complex tax filings and compliance requirements. These customers    │
│  value efficiency, accuracy, and robust reporting tools to meet both regulatory demands and internal auditing   │
│  standards. They typically face challenges such as staying up-to-date with tax law changes, managing multiple   │
│  tax jurisdictions, and reducing the time spent on tax preparation and reporting.                               │
│  .                                                                                                              │
│                                                                                                                 │
│     Let's not spend much time in leads that are not a good fit for us.                                          │
│     Short answers from leads are a yellow flag                                                                  │
│                                                                                                                 │
│     Lead Form Responses:                                                                                        │
│                                                                                                                 │
│  **Form Response**                                                                                              │
│                                                                                                                 │
│  1. **Name of your organization:** Alpha Financial Services Inc.                                                │
│  2. **Contact person's name and designation:** Rachel Green, Tax Compliance Manager                             │
│  3. **Email address and phone number:** rachel.green@alphafinancial.com, (123) 456-7890                         │
│  4. **What is the size of your company?** 500 employees                                                         │
│  5. **How many employees are involved in tax-related processes?** 12 employees                                  │
│  6. **Do you currently use any tax management software? If yes, please specify.** Yes, we currently use         │
│  TaxWise but are exploring alternatives for better reporting and analytics.                                     │
│  7. **What are your biggest challenges in tax compliance and reporting?** Keeping up with changing tax          │
│  regulations across multiple jurisdictions, ensuring accuracy in tax calculations, and reducing the time spent  │
│  on preparing quarterly reports.                       

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Analysis Expert                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  This lead presents a strong opportunity for TaxEdge Solutions.  Here's a breakdown of key insights gleaned     │
│  from their form responses:                                                                                     │
│                                                                                                                 │
│  * **Target Customer Profile:** Alpha Financial Services Inc. aligns perfectly with our ideal customer          │
│  profile. They are a medium to large enterprise (500 employees) with a dedicated tax compliance department,     │
│  indicating a need for robust tax management solutions.                                                         │
│  * **Pain Points:** Their stated challenges – staying current with changing tax regulations, ensuring           │
│  accuracy, and reducing time spent on reporting – are precisely the problems TaxEdge Pro & TaxEdge Analytics    │
│  are designed to solve. The mention of "exploring alternatives" to their current software, TaxWise, signals     │
│  dissatisfaction and a willingness to consider new solutions.                                                   │
│  * **Product Interest:** Expressing interest in both TaxEdge Pro and TaxEdge Analytics is a great indicator.    │
│  This implies a need for comprehensive tax management and also data-driven insights, suggesting they value the  │
│  full suite of our offerings.                                                                                   │
│  * **Proactive Engagement:**  The lead has requested a demo and consultation, indicating active interest and a  │
│  willingness to move forward.                                                                                   │
│                                                                                                                 │
│  **Next Steps:**                                                                                                │
│                                                                                                                 │
│  1. **Prioritize:** This lead should be prioritized for immediate follow-up due to their strong fit and         │
│  demonstrated interest.                                                                                         │
│  2. **Research:**                                                                                               │
│      * Deepen our understanding of Alpha Financial Services' industry, financial size, and specific tax         │
│  jurisdiction challenges through online research.                                                               │
│      * Identify key decision-makers within their tax department to tailor our pitch.                            │
│                                                                                                                 │
│  3. **Tailored Demo:**                                                                                          │
│                                                                                                                 │
│  *  Prepare a demo showcasing how TaxEdge Pro can streamline their compliance processes and address their pain  │
│  points with specific examples relevant to their industry.                                                      │
│  *  Highlight TaxEdge Analytics' capabilities to provid

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4655066e-7346-4c85-98e6-fc812d79a04b                                                                     │
│  Agent: Lead Analysis Expert                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry Research Specialist                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Using the information gathered from the lead's data, conduct preliminary                                   │
│      research on the lead's industry, company background, and potential use                                     │
│      cases for TaxEdge Pro & TaxEdge Analytics.                                                                 │
│      Focus on finding relevant data that can aid in scoring the lead and                                        │
│      planning a strategy to pitch them TaxEdge Pro & TaxEdge Analytics.                                         │
│                                                                                                                 │
│      Let's not spend much time in leads that are not a good fit for us.                                         │
│      Short answers from leads are a yellow flag.                                                                │
│                                                                                                                 │
│      Lead Form Responses:                                                                                       │
│                                                                                                                 │
│  **Form Response**                                                                                              │
│                                                                                                                 │
│  1. **Name of your organization:** Alpha Financial Services Inc.                                                │
│  2. **Contact person's name and designation:** Rachel Green, Tax Compliance Manager                             │
│  3. **Email address and phone number:** rachel.green@alphafinancial.com, (123) 456-7890                         │
│  4. **What is the size of your company?** 500 employees                                                         │
│  5. **How many employees are involved in tax-related processes?** 12 employees                                  │
│  6. **Do you currently use any tax management software? If yes, please specify.** Yes, we currently use         │
│  TaxWise but are exploring alternatives for better reporting and analytics.                                     │
│  7. **What are your biggest challenges in tax compliance and reporting?** Keeping up with changing tax          │
│  regulations across multiple jurisdictions, ensuring accuracy in tax calculations, and reducing the time spent  │
│  on preparing quarterly reports.                                                                                │
│  8. **Which product are you most interested in: TaxEdge Pro, TaxEdge Analytics, or both?** Both products        │
│  9. **Would you like to schedule a demo or consultation?** Yes, I would like to schedule a demo next week.      │
│                                                                                                                 │
│                                                                                                                 │
│      Product Descriptions:                                                                                      │
│                                                                                                                 │
│  TaxEdge Pro is a comprehensive tax management software

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Scoring and Strategic Planner                                                                      │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Utilize the context and the lead's form response to score the lead.                                        │
│      Consider factors such as industry relevance, company size, specific TaxEdge Pro & TaxEdge Analytics        │
│      use case potential, and readiness for partnership.                                                         │
│      Take into account the wording used on the form response and how lenghty                                    │
│      the response was to score the lead.                                                                        │
│      Based on the score, suggest the most appropriate strategy for the lead.                                    │
│      Integrate this information to outline how TaxEdge Solutions and TaxEdge Pro & TaxEdge Analytics can        │
│      specifically benefit this lead.                                                                            │
│                                                                                                                 │
│      We are more focused on                                                                                     │
│  The ideal customers for TaxEdge Solutions are medium to large enterprises, including accounting firms and      │
│  corporate finance departments, that manage complex tax filings and compliance requirements. These customers    │
│  value efficiency, accuracy, and robust reporting tools to meet both regulatory demands and internal auditing   │
│  standards. They typically face challenges such as staying up-to-date with tax law changes, managing multiple   │
│  tax jurisdictions, and reducing the time spent on tax preparation and reporting.                               │
│                                                                                                                 │
│                                                                                                                 │
│      Let's not spend much time in leads that are not a good fit for us.                                         │
│      Smaller comapnies will want to pay less.                                                                   │
│      Short answers from leads are a yellow flag.                                                                │
│      Let's be a little pessimistic and focus the high scores on deals that are                                  │
│      more likely to close and truly great.                                                                      │
│                                                                                                                 │
│      Lead Form Responses:                                                                                       │
│                                                                                                                 │
│  **Form Response**                                                                                              │
│                                                                                                                 │
│  1. **Name of your organization:** Alpha Financial Services Inc.                                                │
│  2. **Contact person's name and designation:** Rachel Green, Tax Compliance Manager                             │
│  3. **Email address and phone number:** rachel.green@al

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Scoring and Strategic Planner                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Alpha Financial Services Inc. - Lead Evaluation & Strategy                                                  │
│                                                                                                                 │
│  **Lead Score:** 9/10                                                                                           │
│                                                                                                                 │
│  **Summary:** Alpha Financial Services Inc. presents a highly qualified lead with a strong alignment to our     │
│  ideal customer profile, a demonstrated need for our solutions, and proactive engagement.                       │
│                                                                                                                 │
│  **Tailored Value Proposition:**                                                                                │
│                                                                                                                 │
│  TaxEdge Solutions offers a comprehensive suite of tools to empower Alpha Financial Services Inc. to navigate   │
│  the complexities of tax compliance and reporting.                                                              │
│                                                                                                                 │
│  * **Streamlined Compliance:** TaxEdge Pro automates complex tax calculations, ensuring accuracy and            │
│  efficiency across multiple jurisdictions. It integrates seamlessly with payroll systems, simplifying payroll   │
│  tax management and minimizing risk.                                                                            │
│                                                                                                                 │
│  * **Data-Driven Insights:** TaxEdge Analytics unlocks the power of your tax data, providing you with deeper    │
│  insights into financial performance, tax trends, and optimization opportunities.  We can help you identify     │
│  potential deductions, credits, and strategies to minimize tax liabilities.                                     │
│                                                                                                                 │
│  * **Increased Efficiency & ROI:**  By reducing manual processes, eliminating errors, and freeing up valuable   │
│  time, TaxEdge Solutions allows you to focus on strategic initiatives and maximize your return on investment.   │
│                                                                                                                 │
│                                                                                                                 │
│  **Talking Points & Strategy:**                                                                                 │
│                                                                                                                 │
│  * **Initial Email:**  Thank Rachel Green for her interest and confirm her preferred demo date and time.        │
│  Highlight two main pain points from her form response that TaxEdge Solutions can directly address:             │
│      * **"Keeping up with changing tax regulations across multiple jurisdictions."** Emphasize TaxEdge Pro's    │
│  real-time updates and jurisdictional expertise.       

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c1016c13-22dc-46c4-bc4d-3bc9ade7a666                                                                     │
│  Agent: Lead Scoring and Strategic Planner                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e9c46c4b-d65f-4d9c-947a-9aa7af0c38e5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Alpha Financial Services Inc. - Lead Evaluation & Strategy                                    │
│                                                                                                                 │
│  **Lead Score:** 9/10                                                                                           │
│                                                                                                                 │
│  **Summary:** Alpha Financial Services Inc. presents a highly qualified lead with a strong alignment to our     │
│  ideal customer profile, a demonstrated need for our solutions, and proactive engagement.                       │
│                                                                                                                 │
│  **Tailored Value Proposition:**                                                                                │
│                                                                                                                 │
│  TaxEdge Solutions offers a comprehensive suite of tools to empower Alpha Financial Services Inc. to navigate   │
│  the complexities of tax compliance and reporting.                                                              │
│                                                                                                                 │
│  * **Streamlined Compliance:** TaxEdge Pro automates complex tax calculations, ensuring accuracy and            │
│  efficiency across multiple jurisdictions. It integrates seamlessly with payroll systems, simplifying payroll   │
│  tax management and minimizing risk.                                                                            │
│                                                                                                                 │
│  * **Data-Driven Insights:** TaxEdge Analytics unlocks the power of your tax data, providing you with deeper    │
│  insights into financial performance, tax trends, and optimization opportunities.  We can help you identify     │
│  potential deductions, credits, and strategies to minimize tax liabilities.                                     │
│                                                                                                                 │
│  * **Increased Efficiency & ROI:**  By reducing manual processes, eliminating errors, and freeing up valuable   │
│  time, TaxEdge Solutions allows you to focus on strategic initiatives and maximize your return on investment.   │
│                                                                                                                 │
│                                                                                                                 │
│  **Talking Points & Strategy:**                                                                                 │
│                                                                                                                 │
│  * **Initial Email:**  Thank Rachel Green for her interest and confirm her preferred demo date and time.        │
│  Highlight two main pain points from her form response that TaxEdge Solutions can directly address:             │
│      * **"Keeping up with changing tax regulations acr

## Alpha Financial Services Inc. - Lead Evaluation & Strategy 

**Lead Score:** 9/10

**Summary:** Alpha Financial Services Inc. presents a highly qualified lead with a strong alignment to our ideal customer profile, a demonstrated need for our solutions, and proactive engagement. 

**Tailored Value Proposition:**

TaxEdge Solutions offers a comprehensive suite of tools to empower Alpha Financial Services Inc. to navigate the complexities of tax compliance and reporting.

* **Streamlined Compliance:** TaxEdge Pro automates complex tax calculations, ensuring accuracy and efficiency across multiple jurisdictions. It integrates seamlessly with payroll systems, simplifying payroll tax management and minimizing risk.

* **Data-Driven Insights:** TaxEdge Analytics unlocks the power of your tax data, providing you with deeper insights into financial performance, tax trends, and optimization opportunities.  We can help you identify potential deductions, credits, and strategies to minimize tax 

In [10]:
lead_assessment_crew.usage_metrics

UsageMetrics(total_tokens=5665, prompt_tokens=4052, cached_prompt_tokens=0, completion_tokens=1613, successful_requests=3)